In [1]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import locale
import time
from datetime import datetime
from threading import Thread
from tqdm import tqdm

In [4]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [5]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

### object detail
id: int  
screen_name: @usuario  
tweet_count: int  
description: descripcion usuario 
location: ubicacion en desc  
url: url en desc  
listed_count: listas de las cuales forma parte  
created_at: cuenta creada el  
crawled_at: cuando se escrapeo  
favs_count: int  
followers_count: int  
following_count: int
verified: true/false

In [6]:
userDf = pd.DataFrame(columns=[
    'id',
    'screen_name',
    'complete_name',
    'tweet_count',
    'description',
    'location',
    'birthdate',
    'url',
    'listed_count',
    'created_at',
    'crawled_at',
    'favs_count',
    'followers_count',
    'following_count',
    'verified'
])

In [7]:
# toScrap = pd.DataFrame(data=['ElGatoyLaCaja', 'matiNotFound', 'Floxie10', 'realdonaldtrump', 'merotegui'], columns=['screen_name'])
toScrap = pd.read_pickle('targetUsersDataset.pkl')
toScrap.head()

,username
235491971,gsrcaballero
497428913,juannfalopa
1536923558,FabianPereyra87
2300560809,NMoraniB
179471208,Sergioogz


In [8]:
def scrap_user(username):
    pageResponse = requests.get('https://twitter.com/'+username, headers=headersChrome, proxies=proxies)
    pageParsed = BeautifulSoup(pageResponse.text, 'html.parser')
    profileNavItems = {
        'tweets': 0,
        'following': 0,
        'followers': 0,
        'favorites': 0,
        'moments': 0,
        'lists': 0
    }
    for itemName in profileNavItems.keys():
        liSearch = pageParsed.findAll(attrs={'class':'ProfileNav-item--'+itemName})
        if(len(liSearch)>0):
            liValue = liSearch[0].findAll(attrs={'class':'ProfileNav-value'})[0]
            if( 'data-count' in  liValue.attrs.keys()):
                profileNavItems[itemName] = liValue.attrs['data-count']
            else:
                profileNavItems[itemName] = liValue.text
    completeName = pageParsed.findAll('h1', attrs={'class':'ProfileHeaderCard-name'})[0].findAll('a')[0].text
    completeName = completeName.replace(u'\xa0', u' ')
    completeName = completeName.replace(u'\n', ' ')
    bio = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-bio'})[0].text
    bio = bio.replace(u'\xa0', u' ')
    bio = bio.replace(u'\n', ' ')
    isVerified = False
    if(len(pageParsed.findAll(attrs={'class':'ProfileHeaderCard-name'})[0].findAll(attrs={'class':'Icon--verified'}))>0):
        isVerified = True
    userId = int(pageParsed.findAll(attrs={'class':'ProfileNav'})[0].attrs['data-user-id'])
    profileHeaderItems = {
        'location': False,
        'url': False,
        'joinDate': False,
        'birthdate': False
    }
    for itemName in profileHeaderItems.keys():
        liSearch = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-'+itemName})
        if(len(liSearch)>0):
            if('u-hidden' not in liSearch[0].attrs['class']):
                liValue = liSearch[0].findAll(attrs={'class':'ProfileHeaderCard-'+itemName+'Text'})[0].text
                liValue = re.sub(u'\n', ' ', liValue)
                liValue = liValue.strip(' ')
                liValue = liValue.replace(u'\xa0', u' ')
                if(itemName == 'joinDate'):
                    liValue = re.sub(r'Se unió en ', '', liValue)
                    liValue = re.sub(r' de ', ' ', liValue)
                    liValue = datetime.strptime(liValue, '%B %Y')
                    liValue = pd.Timestamp(ts_input=liValue)
                if(itemName == 'birthdate'):
                    liValue = re.sub(r'Fecha de nacimiento ', '', liValue)
                profileHeaderItems[itemName] = liValue
    userDf.loc[userId] = {
        'id': userId,
        'screen_name': username,
        'complete_name': completeName,
        'description': bio,
        'verified': isVerified,
        'crawled_at': pd.Timestamp.now(),
        'location': profileHeaderItems['location'],
        'url': profileHeaderItems['url'],
        'created_at': profileHeaderItems['joinDate'],
        'birthdate': profileHeaderItems['birthdate'],
        'tweet_count': profileNavItems['tweets'],
        'listed_count': profileNavItems['lists'],
        'favs_count': profileNavItems['favorites'],
        'followers_count': profileNavItems['followers'],
        'following_count': profileNavItems['following']

    }

In [12]:
def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in tqdm(id_range):
        store[id] = scrap_user(id)
    return store

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    print('Done')
    return True


threaded_process_range(5, toScrap.username.values)




  0%|                                                                                         | 0/3161 [00:00<?, ?it/s]



  0%|                                                                                         | 0/3161 [00:00<?, ?it/s]




  0%|                                                                                         | 0/3161 [00:00<?, ?it/s]





  0%|                                                                                         | 0/3161 [00:00<?, ?it/s]






  0%|                                                                                         | 0/3161 [00:00<?, ?it/s]




  0%|                                                                               | 1/3161 [00:02<2:03:25,  2.34s/it]


  0%|                                                                               | 1/3161 [00:03<2:58:50,  3.40s/it]





  0%|                                                                               | 1/3161 [00:03<2:58:59,  3.40s/it]






  

Done


True

In [17]:
print(len(userDf))
userDf.head()

3866


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
2300560809,2300560809,NMoraniB,Nicole Morani Brown,593,Gerencia General de @PuertoBsAs Ministerio de ...,"Buenos Aires, Argentina",False,False,0,2014-01-01 00:00:00,2019-06-29 23:04:21.059135,2377,1630,1510,False
1536923558,1536923558,FabianPereyra87,Fabian Pereyra,2532,,False,False,False,0,2013-06-01 00:00:00,2019-06-29 23:04:21.289518,4307,53,284,False
235491971,235491971,gsrcaballero,Gus Caballero,21635,"Papa de Cami y Rochi, FANA de La Academia, PRO...",False,False,False,0,2011-01-01 00:00:00,2019-06-29 23:04:21.030207,5683,1691,1360,False
948739166,948739166,Pedro_d35,Pedro D. Montellanos,786,Las cosas hermosas que tiene la vida son aqu...,argentina,False,False,0,2012-11-01 00:00:00,2019-06-29 23:04:23.704801,346,132,580,False
497428913,497428913,juannfalopa,falopita,2256,entienden lo que escribo quienes saben cómo ví...,"Morón, Argentina",03 de abril,False,0,2012-02-01 00:00:00,2019-06-29 23:04:19.992955,4252,146,130,False


In [18]:
#guardar resultados
import pickle
filename = 'UsersDataset.pkl'
with open(filename, 'wb') as twUsers:
    pickle.dump(userDf, twUsers)